<a href="https://colab.research.google.com/github/tirals88/Numerical-Mathematics-and-Computing/blob/main/Chap8_More%20on%20Linear%20Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

%matplotlib inline

# 8.1 테일러 급수법

편미분방정식을 포함한 응용분야에서는 Sparse matrix를 가지는 큰 선형 시스템이 등장한다.

가우스 소거법은 0인 성분들을 0이 아닌 값들로 채울 수 있는 반면, 반복법은 행렬의 희박 구조를 보존한다.

## LU 분해

먼저 $n \times n$ 선형 연립방정식은 $Ax = b$ 형태를 가지며, 이 때 계수 행렬 A 는 다음의 형태를 가진다.

\begin{equation}
A = \left[ \begin{array}{}
a_{11} & a_{12} & a_{13} & \dots & a_{1n} \\
a_{21} & a_{22} & a_{23} & \dots & a_{1n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & a_{n3} & \dots & a_{nn}
\end{array} \right]
\end{equation}

$A$에 순수 가우스 알고리즘을 적용함으로써 $A$를 단순한 두 행렬의 곱으로 분해할 수 있다.


\begin{equation}
L = \left[ \begin{array}{}
1 &  &  &  &  \\
l_{21} & 1 &  &  &  \\
l_{31} & l_{32} & 1 &  &  \\
\vdots & \vdots & \vdots & \ddots &  \\
l_{n1} & l_{n2} & l_{n3} & \dots & 1
\end{array} \right],
U = \left[ \begin{array}{}
u_{11} & u_{12} & u_{13} & \dots & u_{1n} \\
 & u_{22} & u_{23} & \dots & u_{2n} \\
 &  & u_{33} & \dots & u_{3n} \\
 & &  & \ddots & \vdots \\
 &  &  &  & u_{nn}
\end{array} \right]
\end{equation}

$LU$분해를 하기 위해 전진 소거 과정을 거치고 이 과정에서 여러 개의 $M$ 하삼각행렬을 얻게 된다.

$A$를 상삼각행렬로 만들기 위해 여러 개의 $M$을 곱하게 되면 다음의 식을 얻게 된다.

$$M_{3}M_{2}M_{1}Ax = M_{3}M_{2}M_{1}b$$

이제 전진 소거 과정이 완료되었으며 $M = M_{3}M_{2}M_{1}$을 통해 $LU$분해를 얻을 수 있다.

추가로 $LU$ 분해 구조는 알고리즘에서 0으로 나누는 경우가 없다는 점에 의존한다.
LU 분해를 수행하는 코드로 Doolittle factorization 이 있다.


In [ ]:
# Doolittle factorization
def Doolittle(Arr):
  arrL = np.zeros_like(Arr)
  arrU = np.zeros_like(Arr)
  n = Arr.shape[0]
  for i in range(n):
    arrL[i, i] = 1
    for j in range(i, n):
      if i > 0:
        arrU[i, j] = Arr[i, j] - np.dot(arrL[i, :i], arrU[:i, j])
      if i==0:
        arrU[i, j] = Arr[i, j]

    for k in range(i+1, n):
      if i > 0:
        arrL[k, i] = (Arr[k, i] - np.dot(arrL[k, :i], arrU[:i, i])) / arrU[i, i]

      if i==0:
        arrL[k, i] = Arr[k, i]/arrU[i,i]
    #print('i = ', i, '\n', arrL, '\n', arrU)
  return arrL, arrU

tempa = np.array([[6, -2, 2, 4], [12, -8, 6, 10], [3, -13, 9, 3], [-6, 4, 1, -18]]).astype(np.float32)
[tempaL, tempaU] = Doolittle(tempa)
print(tempaL)
print(tempaU)
print(np.dot(tempaL, tempaU) == tempa)

[[ 1.   0.   0.   0. ]
 [ 2.   1.   0.   0. ]
 [ 0.5  3.   1.   0. ]
 [-1.  -0.5  2.   1. ]]
[[ 6. -2.  2.  4.]
 [ 0. -4.  2.  2.]
 [ 0.  0.  2. -5.]
 [ 0.  0.  0. -3.]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


$A$의 $LU$ 분해를 얻을 수 있다면, 시스템 $Ax = b$를 다음과 같이 써서 풀 수 있다.

$$LUx = b$$
위 식은 다시 다음과 같이 변형된다.

\begin{equation}
\begin{cases}
Lz = b\\
Ux = z
\end{cases}
\end{equation}

이 방법은 삼각 시스템을 푸는 것이기 때문에 더 간단하다.

이 둘을 풀기 위해 다음과 같은 코드를 만들 수 있다.

In [ ]:
# 전진 치환 유사코드
# (Unit) Lower triangular system
def Lower_tri(arrL, given_b):
  vec_z = np.zeros_like(given_b)
  vec_z[0] = given_b[0]
  for i in range(1, len(given_b)):
    vec_z[i] = given_b[i] - np.dot(arrL[i, :i], vec_z[:i])

  return vec_z

In [ ]:
# 역대입 유사코드
# Upper triangular system
def Upper_tri(arrU, vec_z):
  sol_x = np.zeros_like(vec_z)
  sol_x[-1] = vec_z[-1]/arrU[-1, -1]
  for i in range(len(vec_z)-2, -1, -1):
    sol_x[i] = (vec_z[i] - np.dot(arrU[i, i+1:], sol_x[i+1:]))/arrU[i, i]

  return sol_x

In [ ]:
# solution = [3, 1, -2, 1]
tempb = np.array([16., 26., -19., -34.])
tempaz = Lower_tri(tempaL, tempb)
tempx = Upper_tri(tempaU, tempaz)
tempx

array([ 3.,  1., -2.,  1.])

## $LDL^{T}$ 분해

주어진 계수행렬 $A$가 대칭적이고, 보통의 $LU$ 분해를 가질 때, $LDL^{T}$ 분해가 수행될 수 있다.

$$LU = A = A^{T} = (LU)^{T} = U^{T}L^{T}$$

$L$은 단위 하삼각행렬이므로 역행렬을 가지며 $U = L^{-1}U^{T}L^{T}$로 쓸 수 있다. 이 식을 다시 변형시키면 다음을 얻을 수 있다.

$$U(L^{T})^{-1} = L^{-1}U^{T}$$

이 때, 좌변은 ***상삼각행렬***, 우변은 ***하삼각행렬***이므로 모두 대각행렬이 된다. 이렇게 만들어진 대각행렬을 $D$라 하면 최종적으로 다음을 얻을 수 있다.

$$A = LU = LDL^{T}$$

$D = \{d_{ii}\}$를 얻기 위해 다음의 식으로 유도할 수 있다.

\begin{eqnarray}
a_{ij} &= &\sum^{n}_{\nu = 1}\sum^{n}_{\mu = 1}{\ell_{i\nu}d_{\nu \mu}(\ell^{T})_{\mu j}} \\
&= &\sum^{n}_{\nu = 1}{\ell_{i\nu}d_{\nu}\ell_{j \nu}}\\
&= &\sum^{min(i, j)}_{\nu = 1}{\ell_{i\nu}d_{\nu}\ell_{j \nu}}\\
\end{eqnarray}

위 식은 $j > i$ 일 때, $\ell_{ij} = 0, \ell_{ii} = 1$ 이기 때문에 가능하다.

다시 $j \leq i$ 라 가정하자, 그러면 다음이 성립한다.

\begin{eqnarray}
a_{ij} &= &\sum^{j}_{\nu = 1}{\ell_{i\nu}d_{\nu}\ell_{j \nu}}\\
&= &\sum^{j-1}_{\nu = 1}{\ell_{i\nu}d_{\nu}\ell_{j \nu}} + \ell_{ij}d_{j}\ell_{j j} \\
&= &\sum^{j-1}_{\nu = 1}{\ell_{i\nu}d_{\nu}\ell_{j \nu}} + \ell_{ij}d_{j}
\end{eqnarray}

특히, $j=i$ 라 하면, 다음을 얻는다.

\begin{eqnarray}
a_{ii} &= &\sum^{i-1}_{\nu = 1}{d_{\nu}\ell^{2}_{i \nu}} + \ell_{ii}d_{i} \\
d_{i} &= &a_{ii} - \sum^{i-1}_{\nu = 1}{d_{\nu}\ell^{2}_{i \nu}}
\end{eqnarray}


또한 $\ell_{ii} = 1$ 임을 알고 있기 때문에 앞선 식으로 $\ell_{ij}$ 를 구할 수 있다.

\begin{eqnarray}
a_{ij} &= &\sum^{j-1}_{\nu = 1}{\ell_{i\nu}d_{\nu}\ell_{j \nu}} + \ell_{ij}d_{j} \\
&\downarrow \\
\ell_{ij} &= &[a_{ij} - \sum^{j-1}_{\nu = 1}{\ell_{i\nu}d_{\nu}\ell_{j \nu}}] / d_{j}
\end{eqnarray}

먼저 $j = 1$ 로 놓으면 $L$ 의 첫 번째 열을 생성하는 식을 얻는다.

$$\ell_{i1} = a_{i1}/d_{1}$$

다음으로 $j = 2$로 놓으면 $L$ 의 두 번째 열을 생성하는 식을 얻는다.

$$\ell_{i2} = (a_{i2} - \ell_{i1}d_{1}\ell_{21})/d_{2}$$

In [48]:
# LDL^T decomposition
# input Arr 또는 함수 내 temp array 들을 실수형으로 지정해줘야함
def LDL_dcomp(Arr):
  LowA = np.zeros_like(Arr).astype('float32')
  arrd = np.zeros(len(Arr)).astype('float32')
  for j in range(len(Arr)):
    LowA[j, j] = 1
    arrd[j] = Arr[j, j] - np.dot(arrd[:j], (LowA[j, :j])**2)
    for i in range(j+1, len(Arr)):
      LowA[j, i] = 0
      temp = ((Arr[i, j] - np.dot((arrd[:j]*LowA[j, :j]), LowA[i, :j])) / arrd[j])
      LowA[i, j] = temp
      #print(((Arr[i, j] - np.dot((arrd[:j]*LowA[j, :j]), LowA[i, :j])) / arrd[j]))

  return LowA, arrd

In [49]:
tempsym = np.array([[4,3,2,1],[3,3,2,1],[2,2,2,1],[1,1,1,1]])
LDL_dcomp(tempsym)

(array([[1.        , 0.        , 0.        , 0.        ],
        [0.75      , 1.        , 0.        , 0.        ],
        [0.5       , 0.6666667 , 1.        , 0.        ],
        [0.25      , 0.33333334, 0.5       , 1.        ]], dtype=float32),
 array([4.       , 0.75     , 0.6666666, 0.5      ], dtype=float32))

$A$가 대칭행렬일 때, $LU$ 분해를 넘어, $LDL^{T}$ 분해를 할 수 있었다. 만약 $A$ 가 추가로 양의 정부호행렬, ***positive definite matrix*** 라면, 촐레스키 분해 Cholesky factorization $A = LL^{T}$ 를 갖는다.

- 대칭 양의 정부호행렬 / ***positive definite matrix*** : $\quad$ 0이 아닌 벡터 $x$ 에 대해, $ x^{T}Ax > 0$ 인 행렬

$A = LDL^{T}$ 일 때, $\widetilde{D} = D^{1/2}$ (대각 성분이 $\sqrt{d_{ii}}$ 인 대각행렬)를 정의하면, $\widetilde{L} \equiv LD^{1/2}$ 이라 할 수 있다. 그리고 다음이 성립한다.

$$A = \widetilde{L}\widetilde{L}^{T}$$

## 다수의 우변

선형 시스템을 풀 때, 다수의 우변 입력을 사용하는 경우가 많다. $B$가 다음과 같이 주어졌다고 가정하자.

\begin{eqnarray}
B &= &[b^{(1)}, b^{(2)}, \dots, b^{(m)}] \\
Ax^{(j)} & = &b^{(j)}
\end{eqnarray}

그리고 이는 간단하게 $AX = B$ 로 쓸 수 있다.

이 때의 연산 횟수는 $A$ 의 분해 과정에서 $\frac{1}{3}n^{3}$ 번, 각각의 $x^{(j)}$ 역대입 과정에서도 $n^{2}$ 번의 긴 연산이 필요하다. 전체 과정에서는 $(\frac{1}{3}n^{3} + mn^{2})$ 번의 긴 연산이 요구된다. 그렇지만 이는 각각의 시스템을 개별적으로 풀 때의 연산 횟수인 $m(\frac{1}{3}n^{3} + n^{2})$ 번보다는 훨씬 적은 수이다.

## $A^{-1}$ 계산

다수의 우변을 적용하면 역행렬을 다음과 같이 표현할 수 있다.

\begin{eqnarray}
AX &= &I \\
Ax^{(j)} & = &I^{(j)}
\end{eqnarray}

추가로 선형 시스템 $Ax=b$를 풀 때, $A^{-1}$을 계산 후에 $x = A^{-1}b$ 를 계산하는 방식은 불필요한 계산을 많이 요구하기 때문에 바람직하지 않다.

위에서 다루었던 $LU$ 분해와 치환행렬 ***permutation matrix*** 를 이용하면 더 간단한 연산들로 해를 구할 수 있다.

먼저 $PA$ 의 $LU$ 분해를 가지고 있다면, 다음과 같이 나타낼 수 있다.

\begin{eqnarray}
\begin{cases}
Ly &= &Pb \\
Ux & = &y
\end{cases}
\end{eqnarray}

# 8.2 고윳값과 고유벡터

$$Ax = \lambda x$$

- ' 도전적인 고윳값 문제에 직면한 사람에게 최고의 조언은 LAPACK 의 소프트웨어를 사용하라는 것이다. ' - 교재 431p.

## 고윳값의 성질

다음의 명제들은 임의의 정사각행렬 $A$ 에 대해서 참이다.

1. $\lambda$ 가 $A$의 고윳값이라면 임의의 다항식 $p$에 대해서 $p(\lambda)$ 는 $p(A)$ 의 고윳값이다. 특히 $\lambda^{k}$ 는 $A^{k}$ 의 고윳값이다.

2. $A$ 가 역행렬을 가지고 $\lambda$ 가 $A$의 고윳값이라면 임의의 다항식 $p$ 에 대하여 $p(\lambda)$ 은 $p(A^{-1})$의 고윳값이다. 특히 $\lambda^{-1}$ 은 $A^{-1}$ 의 고윳값이다.

3. $A$ 가 실수 행렬이자 대칭행렬이면 이 행렬의 고윳값은 실수이다.

4. $A$ 가 복소 행렬이자 헤르미트 Hermite  행렬이면 이 행렬의 고윳값은 실수이다.

5. $A$ 가 헤르미트 행렬이자 양의 정부호행렬이면 이 행렬의 고윳값은 양수이다.

6. $P$ 가 역행렬을 가지면 $A$ 와 $PAP^{-1}$ 은 동일한 특성 다항식(과 동일한 고윳값)을 가진다.

---
***Hermite matrix 란 자기 자신과 켤례 전치가 같은 복소수 정사각행렬이다. 대칭행렬의 일반화이다.***

$${\displaystyle A_{ij}={\overline {A_{ji}}}}$$

먼저 $P$ 가 역행렬을 가지고 $B = PAP^{-1}$을 만족하면 두 행렬은 서로 닮았다고 한다. 그리고 닮은 행렬들은 같은 특성방정식을 갖는다.

\begin{eqnarray}
Det(B - \lambda I) &= &Det(PAP^{-1} - \lambda I)\\
&= &Det(PAP^{-1} - P(\lambda I)P^{-1})\\
&= &Det(P(A-\lambda I)P^{-1}) \\
&= &Det(P) \cdot Det(A-\lambda I)\cdot Det(P^{-1}) \\
&= &Det(A-\lambda I)
\end{eqnarray}